In [3]:
from docquery import document, pipeline
p = pipeline('document-question-answering')

# using shorter PDF for quick testing
doc = document.load_document("../data-generator/notes/Student_Notes_short.pdf")

# use full pdf for real testing
# doc = document.load_document("../data-generator/notes/Student Notes.pdf")

In [4]:
questions = [
  "What are boolean logic operations?", 
  "What is overflow?",
  ]

for q in questions:
  print(q, p(question=q, **doc.context))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
What are boolean logic opera